# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'lxml')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
 

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10


In [3]:
df = pd.DataFrame()
df["reviews"] = reviews

In [4]:
data = df.copy()

In [5]:
data

,reviews
0,✅ Trip Verified | That was supposed to be my ...
1,✅ Trip Verified | Have no fear when your BA f...
2,Not Verified | We have flown BA five times fr...
3,✅ Trip Verified | London Heathrow to Istanbul...
4,"Not Verified | Jan 30th, I booked a last-minut..."
...,...
995,✅ Trip Verified | London Heathrow to Madrid. I...
996,✅ Trip Verified | Gatwick to St Lucia return....
997,✅ Trip Verified | It's been many years since ...
998,✅ Trip Verified | \r\nLondon Heathrow / Pisa r...


In [6]:
data['reviews'] = data.reviews.str.replace("✅ Trip Verified |", '')
data['reviews'] = data.reviews.str.replace("Not Verified | ", '')

In [8]:
df

,reviews
0,✅ Trip Verified | That was supposed to be my ...
1,✅ Trip Verified | Have no fear when your BA f...
2,Not Verified | We have flown BA five times fr...
3,✅ Trip Verified | London Heathrow to Istanbul...
4,"Not Verified | Jan 30th, I booked a last-minut..."
...,...
995,✅ Trip Verified | London Heathrow to Madrid. I...
996,✅ Trip Verified | Gatwick to St Lucia return....
997,✅ Trip Verified | It's been many years since ...
998,✅ Trip Verified | \r\nLondon Heathrow / Pisa r...


In [9]:
df[df['reviews'].str[:1]!='N'] = True
df[df['reviews'].str[:1]=='N'] = False

In [10]:
df.rename(columns={'reviews': 'verified'})

,verified
0,True
1,True
2,False
3,True
4,False
...,...
995,True
996,True
997,True
998,True


In [11]:
data['verified'] = df

In [12]:
from nltk.corpus import stopwords
import re

In [13]:
def clean_text(text):
    punct = r'[^a-zA-Z0-9\s]'   
    stopwords_list = set(stopwords.words('english'))

    text = text.lower()
    text = re.sub(punct, '', text)
    
    final_text = []
    
    for i in text.split():
        if i.strip().lower() not in stopwords_list and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)

In [14]:
data['reviews'] = data['reviews'].apply(clean_text)
data['headers'] = data['headers'].apply(clean_text)

In [15]:
data

,reviews,verified,headers
0,supposed flight cancelled hours takeoff nearly...,True,better worst low cost
1,fear ba flight operated finnair business class...,True,could better
2,flown ba five times western usa england always...,False,never
3,london heathrow istanbul start halfterm school...,True,low ba fallen
4,jan booked lastminute transatlantic flight ire...,False,terrible experience
...,...,...,...
995,london heathrow madrid travel lhr regularly br...,True,service good quality
996,gatwick st lucia return recently returned two ...,True,effectively low cost airline
997,many years since flew long haul british airway...,True,pleasant flight
998,london heathrow pisa return wife fly route tim...,True,miserable rip


In [16]:
data.to_csv("BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.